In [ ]:
"""
Real-Time Earthquake Prediction Script using Trained 1D CNN Model

This script continuously queries live infrasound data, preprocesses it into a 
Power Spectral Density (PSD) vector, and performs classification using a 
pre-trained 1D CNN model to detect potential earthquakes.

Usage:
- Ensure `mean.npy`, `std.npy`, and `CNNmodel.pth` are available in the expected paths.
- The model expects input vectors of length 572 (preprocessed PSD features).
- The script logs predictions to 'earthquake_predictions_log.csv' once every 60 seconds.
- Only predictions where the model detects an earthquake (class 1) are logged.

Requirements:
- PyTorch, NumPy, and access to the `live_stream_query_for_model()` utility.
- A valid CNN model class (`EarthquakeCNN`) and its trained weights.

Outputs:
- Console: Prints predicted class and softmax probabilities.
- File: Logs timestamped predictions (class 1 only) to CSV.

To stop:
- Use Ctrl+C or kill the process to terminate the infinite loop cleanly.
"""

import time
import numpy as np
import torch
import csv
import os
from datetime import datetime, timedelta, UTC
from cnn_model import EarthquakeCNN
from DataQueryUtils import live_stream_query_for_model

# Load normalization stats
mean = np.load("../DataCollection_Preprocessing/Exported_Paros_Data/mean.npy")
std = np.load("../DataCollection_Preprocessing/Exported_Paros_Data/std.npy")

# Load trained model
model = EarthquakeCNN(input_length=572)
model.load_state_dict(torch.load("../ModelTraining/fold_outputs/fold_5/CNNmodel.pth", map_location="cpu"))
model.eval()
model.to('cpu')

# CSV logging setup (create file and header if it doesn't exist)
dir = "LoggedData"
os.makedirs(dir, exist_ok=True)
log_path = "LoggedData/earthquake_predictions_log.csv"
with open(log_path, mode='a', newline='') as f:
    writer = csv.writer(f)
    f.seek(0, 2)  # Move to end of file
    if f.tell() == 0:
        writer.writerow(["timestamp", "window_start", "window_end", "predicted_class", "prob_earthquake", "prob_background"])

try:
    while True:
        query_time = datetime.now(UTC)
        window_end = query_time
        window_start = window_end - timedelta(seconds=60)

        psd_vector = live_stream_query_for_model(mean=mean, std=std)
        if psd_vector is not None:
            input_tensor = torch.tensor(psd_vector, dtype=torch.float32).unsqueeze(0)
            with torch.no_grad():
                output = model(input_tensor)
                probs = torch.softmax(output, dim=1).numpy().flatten()
                pred = np.argmax(probs)

                print(f"Predicted class: {pred} | Probabilities: {probs}")

                # Log only if predicted class is earthquake (class 1)
                if pred == 1:
                    with open(log_path, mode='a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            query_time.isoformat(timespec='seconds'),
                            window_start.isoformat(timespec='seconds'),
                            window_end.isoformat(timespec='seconds'),
                            int(pred),
                            round(float(probs[1]), 5),
                            round(float(probs[0]), 5)
                        ])
        else:
            print("No PSD vector returned from live_stream_query_for_model.")

        time.sleep(60)

except KeyboardInterrupt:
    print("Program terminated by user. Exiting gracefully.")


Starting live_stream_query_for_model...
[INFO] Querying data from 2025-08-07T00:10:22 to 2025-08-07T00:11:22
[EXCEPTION] Error during stream: '_measurement'
No PSD vector returned from live_stream_query_for_model.
Program terminated by user. Exiting gracefully.
